In [8]:
list_config_1 = ["+", "+", "-", "-"]
list_config_2 = ["+", "+", "+", "+"]
list_config_3 = ["-", "-", "-", "-"]
list_config_4 = ["+", "+", "+", "-"]
list_config_5 = ["+", "-", "-", "-"]

In [9]:
#write permutations of 0,1,2,3
def permute(list_config):
    if len(list_config) == 0:
        return []
    if len(list_config) == 1:
        return [list_config]
    l = []
    for i in range(len(list_config)):
        m = list_config[i]
        remLst = list_config[:i] + list_config[i+1:]
        for p in permute(remLst):
            l.append([m] + p)
    return l

In [10]:
index = permute([0,1,2,3])

In [11]:
set_ = [] 

for set_of_index in index:
    string = ''
    for i in set_of_index:
        string += list_config_1[i] 
    set_.append(string)

In [12]:
strings = set(set_)
strings

{'++--', '+-+-', '+--+', '-++-', '-+-+', '--++'}

In [13]:
def find_interacting_pairs(index, string):
    neibours = {'0':[1, 2, 1, 2], '1':[0, 3, 0, 3], '2':[0, 3, 0, 3], '3':[1, 2, 1, 2]}
    neibours = neibours[str(index)]
    interacting_pairs = [(string[index], string[x]) for x in neibours]
    return interacting_pairs

In [14]:
test = find_interacting_pairs(0, '+-++')
test

[('+', '-'), ('+', '+'), ('+', '-'), ('+', '+')]

In [15]:
def find_energy_given_string_of_configurations(string):
    """
    notice we are taking into account the periodic boundary conditions
    """
    energy = 0
    for i in range(len(string)):
        interacting_pairs = find_interacting_pairs(i, string)
        for a,b in interacting_pairs:
            a += '1'
            b += '1'
            prod = a + '*' +b
            energy += eval(prod)
    return (-1/2)*energy ## notice the minus sign and that we count each pair twice

In [16]:
find_energy_given_string_of_configurations('+---')

-0.0

In [17]:
strings

{'++--', '+-+-', '+--+', '-++-', '-+-+', '--++'}

In [19]:
ising_model(strings)

{'++--': -0.0,
 '+-+-': -0.0,
 '-++-': 8.0,
 '-+-+': -0.0,
 '--++': -0.0,
 '+--+': 8.0}

In [20]:
#### scaling the problem

In [48]:
## Generate random configurations of N = L X L spins
import random 
import numpy as np
def generate_random_configurations(L):
    """
    L is the length of the square lattice
    """
    configurations = np.zeros((L, L))
    for i in range(L):
        for j in range(L):
            configurations[i, j] = random.choice([+1, -1])
    return configurations

In [128]:
configuration = generate_random_configurations(2)
configuration 


array([[ 1.,  1.],
       [-1., -1.]])

In [113]:
len(configuration.ravel())

4

In [144]:
def find_interacting_pairs(index, raveled_config, L):
    side_neibours = [index - 1, index + 1]
    top_neibours = [index - L, index + L]
    neibours = side_neibours + top_neibours
    print(neibours)
    interacting_pairs = [(raveled_config[index], raveled_config[x]) for x in neibours]
    return interacting_pairs

In [147]:
## seems to work
test = find_interacting_pairs(2, configuration.ravel(), 2)
test

[1, 3, 0, 4]


IndexError: index 4 is out of bounds for axis 0 with size 4

In [137]:
def find_energy_given_array_of_configurations(configuration, L):
    """
    notice we are taking into account the periodic boundary conditions
    """
    energy = 0
    raveled_config = configuration.ravel()
    ## this is dumb now but will be useful later when we select only one term to flip

    for i in range(len(raveled_config)):
        ## i will be the index of the term. k,l will be the coordinates of the term

        interacting_pairs = find_interacting_pairs(i, raveled_config, L)
        for a, b in interacting_pairs:
            energy += a*b
    return (-1/2)*energy ## notice the minus sign and that we count each pair twice

In [138]:
# plot a 2d grid of the L x L spins
import matplotlib.pyplot as plt
def plot_configurations(configuration, L):
    """
    configurations is a 2d array of the L x L spins
    """
    energy = find_energy_given_array_of_configurations(configuration, L)
    plt.imshow(configuration, cmap = 'Purples', interpolation = 'nearest')
    plt.title(f'Energy = {energy}')

    plt.show()



In [139]:
plot_configurations(configuration, 2)

0
1
2


IndexError: index 4 is out of bounds for axis 0 with size 4